In [11]:
import numpy as np
import pandas as pd
import webhoseio
from aylienapiclient import textapi
import re
from urllib.parse import urlparse

In [12]:
# Ashwin's key - we only get 1000 free requests a month, so be cautious with the number of times you perform queries
webhoseio.config(token="456af32b-3c58-455d-b9f8-90875bfc8f58") 

In [13]:
client = textapi.Client("4657dfa6", "bbfd5faa23a1020d92e946dd83bce6b7")

In [14]:
webhoseio.query?

In [28]:
output_ex = webhoseio.query("filterWebData", {"q":u"thread.title:(Uber\ isn\'t\ sure\ if\ it\ can\ \'remain\ a\ viable\ business\'\ without\ building\ self\-driving\ cars) site:businessinsider.com"})

In [29]:
output_ex

{'moreResultsAvailable': 0,
 'next': '/search?token=456af32b-3c58-455d-b9f8-90875bfc8f58&format=json&ts=1493611653000&q=thread.title%3A%28Uber%5C+isn%27t%5C+sure%5C+if%5C+it%5C+can%5C+%27remain%5C+a%5C+viable%5C+business%27%5C+without%5C+building%5C+self%5C-driving%5C+cars%29+site%3Abusinessinsider.com',
 'posts': [],
 'requestsLeft': 951,
 'totalResults': 0}

## Get the title and site_url of an article given a url

In [17]:
def getTitleAndSiteUrl(url):
    extract = client.Extract({"url": url, "best_image": False})
    title = re.escape(extract['title'])
    site_url = re.sub('www.', '', urlparse(url).hostname)
    return title, site_url

getTitleAndSiteUrl('https://www.google.com/')

('Report\\:\\ Sally\\ Yates\\ will\\ deliver\\ a\\ forceful\\ public\\ rebuke\\ of\\ the\\ Trump\\ administration\\ on\\ Michael\\ Flynn',
 'businessinsider.com')

## Create the dataframe with the 20 features that are in the Kaggle Data Set. Also create the function that will add rows to the dataframe

In [18]:
# all the columns from the Kaggle data set
cols = ["uuid", "ord_in_thread", "author", "published", "title", "text", "language", "crawled", "site_url", "country", "domain_rank", "thread_title", "spam_score", "main_img_url", "replies_count", "participants_count", "likes", "comments", "shares", "type"]
df = pd.DataFrame(columns = cols)
df

,uuid,ord_in_thread,author,published,title,text,language,crawled,site_url,country,domain_rank,thread_title,spam_score,main_img_url,replies_count,participants_count,likes,comments,shares,type


In [19]:
def addRow(df, site_url, title):
    output = webhoseio.query("filterWebData", {"q":"thread.title:(" + title + ") site:" + site_url})
    post = output["posts"][0]
    print(output)
    thread = post["thread"]
    social = thread["social"]["facebook"]
    
    uuid = post["uuid"]
    ord_in_thread = post["ord_in_thread"]
    author = post["author"]
    published = post["published"]
    title = post["title"]
    text = post["text"]
    language = post["language"]
    crawled = post["crawled"]
    site_url = thread["site"]
    country = thread["country"]
    domain_rank = thread["domain_rank"]
    thread_title = thread["title"]
    spam_score = thread["spam_score"]
    img = thread["main_image"]
    replies = thread["replies_count"]
    participants = thread["participants_count"]
    likes = social["likes"]
    comments = social["comments"]
    shares = social["shares"]
    
    #data = [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0,0,0,0,0,0,0,0,0,0,0]]
    data = [[uuid, ord_in_thread, author, published, title, text, language, crawled, site_url, country, domain_rank, thread_title, spam_score, img, replies, participants, likes, comments, shares, "N/A"]]
    df_row = pd.DataFrame(data, columns=cols)
    #print(df_row)
    df = df.append(df_row, ignore_index=True)
    return df

In [20]:
title, site_url = getTitleAndSiteUrl('http://www.breitbart.com/national-security/2017/05/02/north-korea-economy-not-doing-pretty-well/')
print(title)
print(site_url)

df = addRow(df, site_url, title)
df

Vox\ Claims\ North\ Korea\'s\ Economy\ Doing\ \'Pretty\ Well\'
breitbart.com
{'posts': [{'thread': {'title': 'Vox Claims North Korea’s Economy Doing ‘Pretty Well’', 'social': {'pinterest': {'shares': 3}, 'linkedin': {'shares': 2}, 'facebook': {'comments': 0, 'likes': 126, 'shares': 126}, 'gplus': {'shares': 0}, 'vk': {'shares': 0}, 'stumbledupon': {'shares': 0}}, 'url': 'http://omgili.com/ri/jHIAmI4hxg9bm_w_dC_byE8kjeeQbIOUDXhe_L3psUmT9okT02FFmsuJTsOp0uwxHhMo8o1qC03jNtlx2Bya6cXzY5lcYEQvRyM0QB7QRjW2TG3bGAcKcGmJ1TN2UFq59ZncVdpecGE-', 'main_image': None, 'spam_score': 0.0, 'site_section': 'http://www.breitbart.com', 'site': 'breitbart.com', 'performance_score': 1, 'site_full': 'www.breitbart.com', 'title_full': 'Vox Claims North Korea’s Economy Doing ‘Pretty Well’', 'replies_count': 0, 'participants_count': 1, 'site_type': 'blogs', 'country': 'US', 'published': '2017-05-03T04:29:26.000+03:00', 'domain_rank': 967, 'section_title': 'Breitbart News', 'uuid': '38f87e33973a01fbd5fca7b58254331a

,uuid,ord_in_thread,author,published,title,text,language,crawled,site_url,country,domain_rank,thread_title,spam_score,main_img_url,replies_count,participants_count,likes,comments,shares,type
0,38f87e33973a01fbd5fca7b58254331a51402dd7,0.0,John Hayward,2017-05-03T04:29:26.000+03:00,Vox Claims North Korea’s Economy Doing ‘Pretty...,Vox Claims North Korea’s Economy Doing ‘Pretty...,english,2017-05-03T04:29:26.000+03:00,breitbart.com,US,967.0,Vox Claims North Korea’s Economy Doing ‘Pretty...,0.0,None,0.0,1.0,126.0,0.0,126.0,N/A


In [21]:
df = df[2:3]
df.to_pickle('bb_article.pkl')

In [35]:
def get_title_and_site_url(url):
    extract = client.Extract({"url": url, "best_image": False})
    title = re.escape(extract['title'])
    site_url = re.sub('www.', '', urlparse(url).hostname)
    return title, site_url

def extract_article(url):
    title, site_url = get_title_and_site_url(url)
    print(title)
    output = webhoseio.query("filterWebData", {"q":"thread.title:(" + title + ") site:" + site_url})
    print(output)
    post = output["posts"][0]
    thread = post["thread"]
    social = thread["social"]["facebook"]
    uuid = post["uuid"]
    ord_in_thread = post["ord_in_thread"]
    author = post["author"]
    published = post["published"]
    title = post["title"]
    text = post["text"]
    language = post["language"]
    crawled = post["crawled"]
    site_url = thread["site"]
    country = thread["country"]
    domain_rank = thread["domain_rank"]
    thread_title = thread["title"]
    spam_score = thread["spam_score"]
    img = thread["main_image"]
    replies = thread["replies_count"]
    participants = thread["participants_count"]
    likes = social["likes"]
    comments = social["comments"]
    shares = social["shares"]
    data = [[uuid, ord_in_thread, author, published, title, text, language, crawled, site_url, country, domain_rank, thread_title, spam_score, img, replies, participants, likes, comments, shares, "N/A"]]
    article_extracted_df = pd.DataFrame(data, columns=cols)
    return article_extracted_df

In [36]:
extract_article('http://www.businessinsider.com/sally-yates-testimony-trump-michael-flynn-2017-5')

Report\:\ Sally\ Yates\ will\ deliver\ a\ forceful\ public\ rebuke\ of\ the\ Trump\ administration\ on\ Michael\ Flynn
{'posts': [{'thread': {'title': 'Report: Sally Yates will deliver a forceful public rebuke of the Trump administration on Michael Flynn', 'social': {'pinterest': {'shares': 1}, 'linkedin': {'shares': 4}, 'facebook': {'comments': 0, 'likes': 150, 'shares': 150}, 'gplus': {'shares': 1}, 'vk': {'shares': 0}, 'stumbledupon': {'shares': 0}}, 'url': 'http://omgili.com/ri/jHIAmI4hxg8v3BmVd24meuL1ei1Bo_4Wh5TwSLsGQigcMwz5XMsj7aGy0F9l5T6L30Rv5YP7YH81VPy47Aqub2LhbyAHToStS3wB7ScKMpc-', 'main_image': 'http://static5.businessinsider.com/image/59090bbafcd8eb1e038b4697-1190-625/report-sally-yates-will-deliver-a-forceful-public-rebuke-of-the-trump-administration-on-michael-flynn.jpg', 'spam_score': 0.0, 'site_section': 'http://feeds.feedburner.com/businessinsider/politics', 'site': 'businessinsider.com', 'performance_score': 1, 'site_full': 'www.businessinsider.com', 'title_full': 'Rep

,uuid,ord_in_thread,author,published,title,text,language,crawled,site_url,country,domain_rank,thread_title,spam_score,main_img_url,replies_count,participants_count,likes,comments,shares,type
0,d698b86b7ef25159c1aef07c5d2528fc966b371b,0,Natasha Bertrand,2017-05-03T01:58:38.000+03:00,Report: Sally Yates will deliver a forceful pu...,print \nSally Yates. Pete Marovich/Getty Image...,english,2017-05-03T01:58:38.000+03:00,businessinsider.com,US,283,Report: Sally Yates will deliver a forceful pu...,0.0,http://static5.businessinsider.com/image/59090...,0,1,150,0,150,N/A
